In [48]:
import numpy as np

class TreeNode:
    def __init__(self, coef = 1):
        self.coef = coef
        self.zero = None
        self.one = None
        
    def __str__(self):
        return str(self.coef)
    
    def __repr__(self):
        return str(self.coef)
        
class Tree:
    def __init__(self):
        self.root = TreeNode()
        self.dim = 0
    
    def add_layer(self):
        # add a TreeNode to the end of the zero branch
        current = self.root
        while current.zero:
            current = current.zero
        current.zero = TreeNode()
        self.dim += 1
        
    def get_layer(self, layer):
        # return all the nodes in the layer
        if layer == 0:
            return [self.root]
        else:
            return self._get_layer(self.root, layer)
        
    def _get_layer(self, node, layer):
        if layer == 0:
            return [node]
        else:
            result = []
            if node.zero:
                result += self._get_layer(node.zero, layer-1)
            else:
                result += [None] * 2 ** layer
            if node.one:
                result += self._get_layer(node.one, layer-1)
            else:
                result += [None] * 2 ** layer
            return result
        
class Circuit(Tree):
    def __init__(self, dim):
        super().__init__()
        for i in range(dim):
            self.add_layer()
    
    def H(self, qubit):
        # apply Hadamard gate to qubit
        layer = self.get_layer(qubit-1)
        for node in layer:
            if node.one:
                node.zero.coef, node.one.coef = (node.zero.coef + node.one.coef)/np.sqrt(2), (node.zero.coef - node.one.coef)/np.sqrt(2)
            else:
                node.zero.coef = node.zero.coef/np.sqrt(2)
                node.one = node.zero
                
    def X(self, qubit):
        # apply X gate to qubit
        layer = self.get_layer(qubit-1)
        for node in layer:
            node.zero.coef, node.one.coef = node.one.coef, node.zero.coef
            
    def Z(self, qubit):
        # apply Z gate to qubit
        layer = self.get_layer(qubit-1)
        for node in layer:
            node.one.coef *= -1
            
    def cx(self, control, target):
        # apply CNOT gate to control and target
        assert control == target - 1 # control and target must be adjacent
        layer = self.get_layer(control-1)
        for node in layer:
            if node.one:
                node.one.zero.coef, node.one.one.coef = node.one.one.coef, node.one.zero.coef
    
    def cz(self, control, target):
        # apply CZ gate to control and target
        assert control == target - 1 # control and target must be adjacent
        layer = self.get_layer(control-1)
        for node in layer:
            if node.one:
                node.one.one.coef *= -1
                
    def probs(self, qubit):
        # return the probability of measuring 0 and 1 on qubit
        if qubit == 0:
            return [1]
        else:
            prev_probs = self.probs(qubit-1)
            layer = self.get_layer(qubit)
            probs = []
            for q in range(len(prev_probs)):
                if layer[2*q]:
                    prob_0 = prev_probs[q] * np.abs(layer[2*q].coef) ** 2
                else:
                    prob_0 = 0
                if layer[2*q+1]:
                    prob_1 = prev_probs[q] * np.abs(layer[2*q+1].coef) ** 2
                else:
                    prob_1 = 0
                probs.append(prob_0)
                probs.append(prob_1)
            return probs
            
    def __str__(self):
        # print the circuit
        layers = []
        for i in range(self.dim+1):
            layers.append(self.get_layer(i))
        return str(layers)

In [49]:
qc = Circuit(3)
print(qc)
qc.H(1)
print(qc)
qc.H(2)
print(qc)
qc.H(3)
print(qc)
print(qc.probs(1))

[[1], [1, None, None], [1, None, None, None, None, None, None], [1, None, None, None, None, None, None, None, None, None, None, None, None, None, None]]
[[1], [0.7071067811865475, 0.7071067811865475], [1, None, None, 1, None, None], [1, None, None, None, None, None, None, 1, None, None, None, None, None, None]]
[[1], [0.7071067811865475, 0.7071067811865475], [0.0, 0.0, 0.0, 0.0], [1, None, None, 1, None, None, 1, None, None, 1, None, None]]
[[1], [0.7071067811865475, 0.7071067811865475], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]
[0.4999999999999999, 0.4999999999999999]
